# OPN 10: Task a asynchronní programování
---

Obsah:
- Task
- async a await

TODO: pridat nahodne exception

## Task

Pomocí třídy `Task` můžeme spouštět typicky asynchronně jednu operaci (metodu) bez návratové hodnoty. 

V následujícím příkladu spustíme provádění metody Metoda pomocí statické metody `Task.Run`, tato metoda nám vrátí proměnnou `task`, krerá popisuje již spuštěnou operaci běžící typicky v jiném vlákně. Kompilátor se totiž může rozhodnout, zda se vyplatí operaci pouštět v jiném vlákně nebo ji stačí provést sychronně ve stejném vlákně. 

Příkaz `task.Wait();` pozastaví běh hlavního vlákna programu.

In [24]:
int DlouhoBeziciMetoda(bool exception)
{
    for (int i = 0; i < 5; i++)
    {
        Console.WriteLine($"Metoda: {i}");
        System.Threading.Thread.Sleep(1000);

        if(exception && Random.Shared.Next() % 5 == 0)
        {
            throw new Exception("Nahodna vyjimka");
        } 
    }

    return 5;
}

void MetodaCekajiciNaVysledek()
{
    try
    {
        int vysledek = DlouhoBeziciMetoda(exception: true);
        Console.WriteLine($"vysledek: {vysledek}");
    }
    catch(Exception ex)
    {
        Console.WriteLine($"{ex.GetType()}: {ex.Message}");
    }
}

MetodaCekajiciNaVysledek();

for (int i = 0; i < 5; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

Console.WriteLine("Konec programu");

Metoda: 0
Metoda: 1
Metoda: 2
Metoda: 3
Metoda: 4
System.Exception: Nahodna vyjimka
Main: 0
Main: 1
Main: 2
Main: 3
Main: 4
Konec programu


In [25]:
Task<int> DlouhoBeziciMetodaAsync(bool exception)
{
    return Task.Run(() =>
    {
        for (int i = 0; i < 5; i++)
        {
            Console.WriteLine($"Metoda: {i}");
            System.Threading.Thread.Sleep(1000);

            if(exception && Random.Shared.Next() % 5 == 0)
            {
                throw new Exception("Nahodna vyjimka");
            } 
        }

        return 5;
    });
}

void MetodaCekajiciNaVysledek()
{
    try
    {
        Task<int> task = DlouhoBeziciMetodaAsync(exception: true);
        
        task.Wait();

        int vysledek = task.Result;

        Console.WriteLine($"vysledek: {vysledek}");
    }
    catch(Exception ex) // https://learn.microsoft.com/en-us/dotnet/standard/parallel-programming/exception-handling-task-parallel-library
    {
        Console.WriteLine($"{ex.GetType()}: {ex.Message}");
    }
}

MetodaCekajiciNaVysledek();

for (int i = 0; i < 5; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

Console.WriteLine("Konec programu");

Metoda: 0
Metoda: 1
Metoda: 2
Metoda: 3
Metoda: 4
System.AggregateException: One or more errors occurred. (Nahodna vyjimka)
Main: 0
Main: 1
Main: 2
Main: 3
Main: 4
Konec programu


In [32]:
Task<int> DlouhoBeziciMetodaAsync(bool exception)
{
    return Task.Run(() =>
    {
        for (int i = 0; i < 5; i++)
        {
            Console.WriteLine($"Metoda: {i}");
            System.Threading.Thread.Sleep(1000);

            if(exception && Random.Shared.Next() % 5 == 0)
            {
                throw new Exception("Nahodna vyjimka");
            } 
        }

        return 5;
    });
}

void MetodaCekajiciNaVysledek()
{
    try
    {
        Task<int> task = DlouhoBeziciMetodaAsync(exception: true);
        
        task.ContinueWith(task => 
        {
            if(task.Status == TaskStatus.RanToCompletion)
            {
                Console.WriteLine($"vysledek: {task.Result}");
            }
            else
            {
                if(task.Status == TaskStatus.Faulted)
                {
                    Console.WriteLine($"Chyba: {task.Exception.Message}");
                }
            }
        });
    }
    catch(Exception ex)
    {
        Console.WriteLine($"{ex.GetType()}: {ex.Message}");
    }
}

MetodaCekajiciNaVysledek();

for (int i = 0; i < 5; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

Console.WriteLine("Konec programu");

Main: 0
Metoda: 0
Main: 1
Chyba: One or more errors occurred. (Nahodna vyjimka)
Main: 2
Main: 3
Main: 4
Konec programu


In [33]:
Task<int> DlouhoBeziciMetodaAsync(bool exception)
{
    return Task.Run(() =>
    {
        for (int i = 0; i < 5; i++)
        {
            Console.WriteLine($"Metoda: {i}");
            System.Threading.Thread.Sleep(1000);

            if(exception && Random.Shared.Next() % 5 == 0)
            {
                throw new Exception("Nahodna vyjimka");
            } 
        }

        return 5;
    });
}

async void MetodaCekajiciNaVysledek()
{
    try
    {
        Task<int> task  = DlouhoBeziciMetodaAsync(exception: true);

        int vysledek = await task;

        Console.WriteLine($"vysledek: {vysledek}");
    }
    catch(Exception ex)
    {
        Console.WriteLine($"{ex.GetType()}: {ex.Message}");
    }
}

MetodaCekajiciNaVysledek();

for (int i = 0; i < 5; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

Console.WriteLine("Konec programu");

Main: 0
Metoda: 0
Main: 1
System.Exception: Nahodna vyjimka
Main: 2
Main: 3
Main: 4
Konec programu


Příkaz `await task;` nepozastaví běh programu, ale jen přeruší provádění metody `Main` a mezití běží ostatní operace a po dokončení metody `Metoda` bude v provádění metody `Main` pokračovat. 

In [ ]:
int DlouhoBeziciMetoda()
{
    for (int i = 0; i < 10; i++)
    {
        Console.WriteLine($"Metoda: {i}");
        System.Threading.Thread.Sleep(1000);
    }

    return 5;
}

void MetodaCekajiciNaVysledek()
{
    int vysledek = DlouhoBeziciMetoda();
    Console.WriteLine($"vysledek: {vysledek}");
}

Task task = Task.Run(DlouhoBeziciMetoda);

Console.WriteLine("Task spusten");

task.Wait();

for (int i = 0; i < 10; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

Console.WriteLine("Konec programu");

In [ ]:
void DlouhoBeziciMetoda()
{
    for (int i = 0; i < 10; i++)
    {
        Console.WriteLine($"Metoda: {i}");
        System.Threading.Thread.Sleep(400);
    }
}

void MetodaCekajiciNaVysledek()
{
    
}

Task task = Task.Run(DlouhoBeziciMetoda);
Console.WriteLine("Task spusten");
await task;

Console.WriteLine("Konec programu");
